In [74]:
import numpy 
import pandas as pd
import tensorflow as tf
from google.colab import files
uploades = files.upload()

Saving Total_Data.xlsx to Total_Data (1).xlsx


In [75]:
df = pd.read_excel('Total_Data.xlsx') 

The above data has all entries in 2003-2019 Premier League

In [76]:
# Gives us the dataframe of team a_vs_team b for all past years
def a_vs_b(a,b):
  options = [a,b]
  a_b = df[df['Home Team'].isin(options)]
  a_b = a_b[a_b['Away Team'].isin(options)]
  return a_b

# Gives us the dataframe of team_vs_all for all past years
def team_vs_all(team):
  options = [team]
  team_all = df[df['Home Team'].isin(options) | df['Away Team'].isin(options)]
  return team_all

# Gives us the dataframe of all the home matches of a team
def home_matches(team):
  options = [team]
  home_all = df[df['Home Team'].isin(options)]
  return home_all

# Gives us the dataframe of all the away matches of a team
def away_matches(team):
  options = [team]
  home_all = df[df['Away Team'].isin(options)]
  return home_all

In [77]:
# Taking example of two teams and then will make the whole code in a general form
a = 'Arsenal' #HomeTeam
b = 'Liverpool' #AwayTeam

In [78]:
a_all = team_vs_all(a) # team a vs all other teams data
b_all = team_vs_all(b) # team b vs all other team data
a_b = a_vs_b(a,b) # team a vs team b data

In [79]:
#If team a wins then Win_Value = 1, if loses then 0, if draw then 0.5  
a_all = pd.DataFrame(a_all)
a_all.loc[((a_all['Home Team'] == a) & (a_all['FTR'] == 'H')) | ((a_all['Away Team'] == a) & (a_all['FTR'] == 'A')), 'Win_Value'] = 1
a_all.loc[((a_all['Home Team'] == a) & (a_all['FTR'] == 'A')) | ((a_all['Away Team'] == a) & (a_all['FTR'] == 'H')), 'Win_Value'] = 0
a_all = a_all.fillna(0.5)

b_all = pd.DataFrame(b_all)
b_all.loc[((b_all['Home Team'] == b) & (b_all['FTR'] == 'H')) | ((b_all['Away Team'] == b) & (b_all['FTR'] == 'A')), 'Win_Value'] = 1
b_all.loc[((b_all['Home Team'] == b) & (b_all['FTR'] == 'A')) | ((b_all['Away Team'] == b) & (b_all['FTR'] == 'H')), 'Win_Value'] = 0
b_all = b_all.fillna(0.5)

a_b = pd.DataFrame(a_b)
a_b.loc[((a_b['Home Team'] == a) & (a_b['FTR'] == 'H')) | ((a_b['Away Team'] == a) & (a_b['FTR'] == 'A')), 'Win_Value_a'] = 1
a_b.loc[((a_b['Home Team'] == a) & (a_b['FTR'] == 'A')) | ((a_b['Away Team'] == a) & (a_b['FTR'] == 'H')), 'Win_Value_a'] = 0
a_b = a_b.fillna(0.5) 

a_b.loc[a_b['Home Team'] == a,'Home Team'] = 1
a_b.loc[a_b['Away Team'] == a,'Away Team'] = 1
a_b.loc[a_b['Home Team'] == b,'Home Team'] = 0
a_b.loc[a_b['Away Team'] == b,'Away Team'] = 0
# Team a = 1 (Arsenal) and team b = 0 (Liverpool)

In [81]:
# Same as above but for home matches data
a_home = home_matches(a)
b_home = home_matches(b)

a_home = pd.DataFrame(a_home)
a_home.loc[((a_home['Home Team'] == a) & (a_home['FTR'] == 'H')), 'Home_Win_Value'] = 1
a_home.loc[((a_home['Home Team'] == a) & (a_home['FTR'] == 'A')), 'Home_Win_Value'] = 0
a_home = a_home.fillna(0.5) 

b_home = pd.DataFrame(b_home)
b_home.loc[((b_home['Home Team'] == b) & (b_home['FTR'] == 'H')), 'Home_Win_Value'] = 1
b_home.loc[((b_home['Home Team'] == b) & (b_home['FTR'] == 'A')), 'Home_Win_Value'] = 0
b_home = b_home.fillna(0.5) 

In [83]:
# Same as above but for away matches data
a_away = away_matches(a)
b_away = away_matches(b)
a_away = pd.DataFrame(a_away)
b_away = pd.DataFrame(b_away)

a_away.loc[((a_away['Away Team'] == a) & (a_away['FTR'] == 'H')), 'Away_Win_Value'] = 0
a_away.loc[((a_away['Away Team'] == a) & (a_away['FTR'] == 'A')), 'Away_Win_Value'] = 1
a_away = a_away.fillna(0.5) 

b_away.loc[((b_away['Away Team'] == b) & (b_away['FTR'] == 'H')), 'Away_Win_Value'] = 0
b_away.loc[((b_away['Away Team'] == b) & (b_away['FTR'] == 'A')), 'Away_Win_Value'] = 1
b_away = b_away.fillna(0.5) 

In [84]:
# It takes average of last 6 win valus of team a in the matches team a vs team b
values = []
win_prob = []
sum = 0

for value in a_b['Win_Value_a']:
    values.append(value)

for i in range(len(values)):
    if i<6:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-6]
        prob = sum / 6
    win_prob.append(prob) 

a_b["Win_last6_a"] = win_prob

In [85]:
# Similar as above but average of last 5 home win value of team a 
values = []
win_prob = []
sum = 0

for value in a_home['Home_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

a_home["Home_Win_Prob"] = win_prob

# Similar as above but average of last 5 home win value of team b
values = []
win_prob = []
sum = 0

for value in b_home['Home_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

b_home["Home_Win_Prob"] = win_prob

# Similar as above but average of last 5 away win value of team a
values = []
win_prob = []
sum = 0

for value in a_away['Away_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

a_away["Away_Win_Prob"] = win_prob

# Similar as above but average of last 5 away win value of team b
values = []
win_prob = []
sum = 0

for value in b_away['Away_Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

b_away["Away_Win_Prob"] = win_prob

In [86]:
# Similar as above but average of last 10 win value of team a vs all teams
values = []
win_prob = []
sum = 0

for value in a_all['Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<10:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-10]
        prob = sum / 10
    win_prob.append(prob) 

a_all["Win_Prob_Last10_a"] = win_prob

# Similar as above but average of last 10 win value of team b vs all teams
values = []
win_prob = []
sum = 0

for value in b_all['Win_Value']:
    values.append(value)
    
for i in range(len(values)):
    if i<10:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-10]
        prob = sum / 10
    win_prob.append(prob) 

b_all["Win_Prob_Last10_b"] = win_prob

# Take average of Home team Shots/Away Team shots average on target for both teams
values = []
win_prob = []
sum = 0

for value in b_all['HSTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

b_all["HSTP_avg"] = win_prob

values = []
win_prob = []
sum = 0

for value in b_all['ASTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

b_all["ASTP_avg"] = win_prob

values = []
win_prob = []
sum = 0

for value in a_all['ASTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

a_all["ASTP_avg"] = win_prob

values = []
win_prob = []
sum = 0

for value in a_all['HSTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    win_prob.append(prob) 

a_all["HSTP_avg"] = win_prob

In [87]:
# Merging data set
data = pd.merge(a_b,a_home[['Match_Id','Home_Win_Prob']], on='Match_Id',how = 'left')
b_home = b_home[['Match_Id','Home_Win_Prob']]
data.loc[data.Match_Id.isin(b_home.Match_Id), ['Home_Win_Prob']] = b_home.loc[b_home.Match_Id.isin(data.Match_Id),['Home_Win_Prob']].values

data = pd.merge(data,a_away[['Match_Id','Away_Win_Prob']], on='Match_Id',how = 'left')
b_away = b_away[['Match_Id','Away_Win_Prob']]
data.loc[data.Match_Id.isin(b_away.Match_Id), ['Away_Win_Prob']] = b_away.loc[b_away.Match_Id.isin(data.Match_Id),['Away_Win_Prob']].values

data = pd.merge(data,a_all[['Match_Id','ASTP_avg']], on='Match_Id',how = 'left')
data.loc[data.Match_Id.isin(b_all.Match_Id), ['ASTP_avg']] = b_all.loc[b_all.Match_Id.isin(data.Match_Id),['ASTP_avg']].values
data = pd.merge(data,a_all[['Match_Id','HSTP_avg']], on='Match_Id',how = 'left')
data.loc[data.Match_Id.isin(b_all.Match_Id), ['HSTP_avg']] = b_all.loc[b_all.Match_Id.isin(data.Match_Id),['HSTP_avg']].values

data = pd.merge(data,a_all[['Match_Id','Win_Prob_Last10_a']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','Win_Prob_Last10_b']], on='Match_Id', how='left')

In [88]:
data.head()

,Home Team,Away Team,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Year,Match_Id,HSTP,ASTP,HP,DP,AP,Win_Value_a,Win_last6_a,Home_Win_Prob,Away_Win_Prob,ASTP_avg,HSTP_avg,Win_Prob_Last10_a,Win_Prob_Last10_b
0,0,1,1,2,A,1,1,D,13,12,7,8,12,20,11,3,2,2,0,0,2.500,3.30,2.500,2003,71,0.538462,0.666667,0.362637,0.274725,0.362637,1.0,1.000000,0.375,0.875,0.561438,0.479240,0.875,0.50
1,1,0,4,2,H,1,2,A,12,12,7,5,16,9,2,8,3,1,0,0,1.909,3.10,4.333,2003,309,0.583333,0.416667,0.486292,0.299462,0.214247,1.0,1.000000,0.900,0.300,0.604094,0.543462,0.950,0.60
2,0,1,2,1,H,1,0,H,11,3,7,2,6,16,6,0,0,2,0,0,3.400,3.25,2.100,2004,527,0.636364,0.666667,0.272836,0.285429,0.441735,0.0,0.666667,0.800,0.500,0.622775,0.537660,0.600,0.55
3,1,0,3,1,H,2,0,H,11,11,8,6,13,10,1,4,0,2,0,0,1.720,3.40,5.000,2004,748,0.727273,0.545455,0.540575,0.273467,0.185958,1.0,0.750000,1.000,0.200,0.624296,0.573232,0.900,0.45
4,0,1,1,0,H,0,0,D,22,6,9,2,12,15,13,1,0,1,0,0,2.100,3.10,3.600,2005,1016,0.409091,0.333333,0.442331,0.299643,0.258026,0.0,0.600000,0.900,0.500,0.481685,0.391818,0.550,0.60


In [89]:
columns = ['Match_Id','Home Team', 'Away Team', 'HSTP_avg', 'ASTP_avg', 'Win_last6_a', 'Home_Win_Prob', 'Away_Win_Prob', 'Win_Prob_Last10_a', 'Win_Prob_Last10_b']
X = data[columns]
X.set_index('Match_Id')

,Home Team,Away Team,HSTP_avg,ASTP_avg,Win_last6_a,Home_Win_Prob,Away_Win_Prob,Win_Prob_Last10_a,Win_Prob_Last10_b
Match_Id,,,,,,,,,
71,0,1,0.479240,0.561438,1.000000,0.375,0.875,0.875,0.50
309,1,0,0.543462,0.604094,1.000000,0.900,0.300,0.950,0.60
527,0,1,0.537660,0.622775,0.666667,0.800,0.500,0.600,0.55
748,1,0,0.573232,0.624296,0.750000,1.000,0.200,0.900,0.45
1016,0,1,0.391818,0.481685,0.600000,0.900,0.500,0.550,0.60
1045,1,0,0.409784,0.479762,0.666667,0.600,0.200,0.500,0.50
1258,1,0,0.618396,0.499780,0.666667,0.800,0.000,0.700,0.45
1443,0,1,0.475714,0.465388,0.500000,0.700,0.500,0.650,0.70
1626,0,1,0.375613,0.487028,0.583333,0.600,0.700,0.900,0.75


In [91]:
data.loc[((data['Home Team'] == 0) & (data['FTR'] == 'H')) | ((data['Away Team'] == 0) & (data['FTR'] == 'A')), 'Win Team'] = 0
data.loc[((data['Home Team'] == 1) & (data['FTR'] == 'H')) | ((data['Away Team'] == 1) & (data['FTR'] == 'A')), 'Win Team'] = 1
data.loc[(data['FTR'] == 'D'), 'Win Team'] = 2 # Labeled value of draw matches for y column

y = data[['Win Team']]